In [1]:
import nltk
import sklearn_crfsuite
import eli5

In [2]:
import pickle

In [4]:
from sklearn.model_selection import train_test_split

In [3]:
with open("word_data_file.obj", "rb") as infile:
    data = pickle.load(infile)

def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]
    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag 
    }
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1
        })
    else:
        features['BOS'] = True
        
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1
        })
    else:
        features['EOS'] = True
                
    return features

# A function for extracting features in documents
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

# A function fo generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]

In [5]:
%%time
X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

CPU times: user 3.81 s, sys: 500 ms, total: 4.31 s
Wall time: 4.39 s


In [18]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.2049587713810429, 
    c2=0.035279048949617219, 
    max_iterations=200, 
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CPU times: user 3min 9s, sys: 4.71 s, total: 3min 14s
Wall time: 3min 24s


In [19]:
eli5.show_weights(crf, top=5, show=['transition_features'])

From \ To,I,date,geogName,orgName,persName,placeName,time
I,2.805,-0.117,0.545,0.91,0.601,0.706,-0.261
date,-0.102,0.87,-1.956,-1.132,-1.5,-1.486,-2.078
geogName,0.563,-1.799,1.734,-0.718,-1.388,-1.418,-1.518
orgName,1.112,-0.798,-0.672,1.937,-1.085,-0.751,-0.661
persName,0.313,-0.855,-2.057,-1.874,0.988,-1.759,-1.821
placeName,0.249,-1.636,-1.652,-1.268,-1.712,0.911,-2.535
time,-0.181,-1.816,-1.507,-1.285,-1.998,-2.569,1.384


In [20]:
eli5.show_weights(crf, top=30)

In [25]:
eli5.show_weights(crf, top=10, feature_re='^word\.is', 
                  horizontal_layout=True, show=['targets'], targets=['date', 'time', 
                                                                     'persName', 'placeName', 
                                                                     'geogName', 'orgName'])

In [26]:
eli5.show_weights(crf, top=10, feature_re='^word\.is', 
                  horizontal_layout=True, show=['targets'], targets=['date', 'time', 
                                                                     'persName'])

In [27]:
eli5.show_weights(crf, top=10, feature_re='^word\.is', 
                  horizontal_layout=True, show=['targets'], targets=['placeName', 
                                                                     'geogName', 'orgName'])

In [28]:
eli5.show_weights(crf, top=10, feature_re='postag', 
                  horizontal_layout=True, show=['targets'], targets=['placeName', 
                                                                     'geogName', 'orgName'])

In [29]:
eli5.show_weights(crf, top=10, feature_re='postag', 
                  horizontal_layout=True, show=['targets'], targets=['date', 'time', 
                                                                     'persName'])